In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd /home/adityasidharta/git/shopee_data_science
%env PROJECT_PATH /home/adityasidharta/git/shopee_data_science

In [ ]:
from utils.envs import *
from model.heuristic.mobile.extractor import *
import pandas as pd
from model.heuristic.mobile.enricher import *

In [ ]:
import json

def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

In [ ]:
Ext = Extractor()
Enr = Enricher()

In [ ]:
train_df = pd.read_csv(mobile_train_repo)
mobile_json = load_json(mobile_profile_json)

In [ ]:
invert_mobile_json = {}
for column in mobile_json.keys():
    invert_mobile_json[column] = {float(k):v for v, k in mobile_json[column].items()}

In [ ]:
train_df.head()

In [ ]:
for column in mobile_json.keys():
    train_df[column] = train_df[column].replace(invert_mobile_json[column])

In [ ]:
train_df.head()

In [ ]:
extracted = Ext.extract_from_title(train_df.at[0, 'title'])
enriched = Enr.enrich(extracted)

In [ ]:
from tqdm import tqdm

In [ ]:
result_list = []
for idx in tqdm(range(len(train_df))):
    extracted = Ext.extract_from_title(train_df.at[idx, 'title'])
    enriched = Enr.enrich(extracted)
    result_dict = {'itemid': train_df.at[idx, 'itemid']}
    for column in mobile_json.keys():
        value = enriched[column]
        if isinstance(value, list):
            if len(value) == 1:
                result_dict[column] = value[0]
            else:
                result_dict[column] = ''
        else:
            result_dict[column] = value
    result_list.append(result_dict)
kyle_df = pd.DataFrame(result_list)

In [ ]:
kyle_column = kyle_df.columns
kyle_column = ['Predict_' + x if x in mobile_json.keys() else x for x in kyle_df.columns]
kyle_df.columns = kyle_column

In [ ]:
merge_df = train_df.merge(kyle_df, on='itemid', how='left')

In [ ]:
for column in mobile_json.keys():
    input_df = merge_df.copy()
    input_df = input_df[~pd.isnull(input_df[column])]
    input_df = input_df[input_df['Predict_' + column] != '']
    print(len(input_df))
    print('{} : {}'.format(column, (input_df[column] == input_df['Predict_' + column]).sum() / len(input_df)))